In [1]:
!pip install git+https://github.com/BindsNET/bindsnet.git

  Cloning https://github.com/BindsNET/bindsnet.git to /tmp/pip-req-build-3kw1jwjk
  Running command git clone --filter=blob:none --quiet https://github.com/BindsNET/bindsnet.git /tmp/pip-req-build-3kw1jwjk
  Resolved https://github.com/BindsNET/bindsnet.git to commit 38c4306a403d9cca47225d069870ac023030276a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 70.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 94.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 18.1 MB/s eta

In [12]:
!pip install torch torchvision

In [20]:
import torch
import torchvision
import torchvision.transforms as transforms
from bindsnet.encoding import PoissonEncoder
from bindsnet.network import Network
from bindsnet.network.nodes import Input, LIFNodes
from bindsnet.network.topology import Connection
from bindsnet.learning import PostPre
from bindsnet.pipeline import EnvironmentPipeline

In [27]:
# Parameters
time = 100  # Time in ms
dt = 1.0  # Time step
input_size = 784  # MNIST images are 28x28 pixels
hidden_size = 100  # Number of neurons in the hidden layer
output_size = 10  # Number of neurons in the output layer
learning_rate = 0.005  # Learning rate
w_init = 0.3  # Initial weight scale

In [34]:

# Load MNIST data using torchvision
transform = transforms.Compose([transforms.ToTensor(), transforms.Lambda(lambda x: x * 128)])
mnist_train = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)

In [30]:
# Create Poisson encoder
encoder = PoissonEncoder(time=time, dt=dt)

# Create a spiking neural network
network = Network(dt=dt)


In [31]:
# Add layers
input_layer = Input(n=input_size, traces=True)
hidden_layer = LIFNodes(n=hidden_size, traces=True, thresh=-52.0, rest=-65.0, reset=-65.0, decay=1e-2)
output_layer = LIFNodes(n=output_size, traces=True, thresh=-52.0, rest=-65.0, reset=-65.0, decay=1e-2)

network.add_layer(input_layer, name='Input')
network.add_layer(hidden_layer, name='Hidden')
network.add_layer(output_layer, name='Output')

In [32]:
# Add connections between layers
input_hidden_conn = Connection(
    source=input_layer, target=hidden_layer, w=w_init * torch.rand(input_size, hidden_size), update_rule=PostPre, nu=[learning_rate, learning_rate]
)
hidden_output_conn = Connection(
    source=hidden_layer, target=output_layer, w=w_init * torch.rand(hidden_size, output_size), update_rule=PostPre, nu=[learning_rate, learning_rate]
)

network.add_connection(input_hidden_conn, source='Input', target='Hidden')
network.add_connection(hidden_output_conn, source='Hidden', target='Output')


In [33]:
# Function to encode data and train the network
def train(network, train_data, encoder, n_epochs=1):
    for epoch in range(n_epochs):
        for step, (image, label) in enumerate(train_data):
            image = image.view(-1)
            encoded_image = encoder(image)
            inputs = {'Input': encoded_image}

            # Run the network for the input time
            network.run(inputs=inputs, time=time)

            # Reset the network state variables
            network.reset_state_variables()

            if step % 100 == 0:
                print(f"Epoch {epoch + 1}/{n_epochs}, Step {step}")

print("Training started...")
train(network, mnist_train, encoder)
print("Training completed.")

Training started...
Epoch 1/1, Step 0
Epoch 1/1, Step 100


KeyboardInterrupt: 